## 前情提要
由於目前 Google Gemini 的 text-embedding-004 的繁體中文支援性不是那麼好。因此本Notebook為了展示繁體中文內容檢索。採用微軟的 `multilingual-e5-large` 搭配 Huggingface 作為示範。

更多繁體中文的模型的檢索能力評比可以參考：
- [使用繁體中文評測各家 Embedding 模型的檢索能力](https://ihower.tw/blog/archives/12167)
- [愛好繁體中文 Embeddings 模型評測 by ihower](https://docs.google.com/spreadsheets/d/1zad1tMFp7OmNjUvm_a-Ni22av2uBmqYclVRgJQGUtl0/edit?gid=0#gid=0)

# Day 2 - Document Q&A with RAG using Chroma (支援繁體中文檢索)
歡迎回來參加 Kaggle 5 天生成式 AI 課程！

**注意**：Day 1 筆記本中包含了許多設定 Kaggle 筆記本的相關資訊。如果你遇到任何問題，請參考[那邊的故障排除步驟](https://www.kaggle.com/code/markishere/day-1-prompting#Get-started-with-Kaggle-notebooks)。

大型語言模型（LLM）有兩個主要的限制：1) 它們僅「知道」訓練期間接觸過的資訊，2) 輸入內容的長度有限。解決這兩個限制的方法之一，就是採用檢索增強生成技術（Retrieval Augmented Generation, RAG）。一個 RAG 系統包含三個階段：

1. 索引建立
2. 文件檢索
3. 回答生成

索引建立是在查詢前預先進行的，可以讓你在查詢時迅速查找相關資訊。當收到查詢後，系統會檢索出相關文件，將文件內容與指示與使用者查詢結合，再利用大型語言模型生成符合查詢需求的自然語言答案。這使得你可以提供模型訓練期間未曾見過的資訊，例如產品專屬知識或即時天氣更新。

在這個筆記本中，你將使用 Gemini API 建立向量資料庫，從資料庫中檢索文件來回答問題，並生成最終答案。你會使用 [Chroma](https://docs.trychroma.com/)，這是一個開源的向量資料庫。利用 Chroma，你可以儲存嵌入向量以及元資料，對文件和查詢進行嵌入，並搜尋文件。

## 幫助

**常見問題請參考** [FAQ 與故障排除指南](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs)。

## 環境設定

首先，請安裝 ChromaDB 與 Gemini API 的 Python SDK。

In [1]:
!pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:0

In [2]:
from google import genai
from google.genai import types

from IPython.display import Markdown

genai.__version__

'1.7.0'

### 資料
以下是一組小型文件資料集，你將用它們來建立嵌入資料庫。


In [3]:
DOCUMENT1 = "操作氣候控制系統  你的 Google 車配備了氣候控制系統，可調整車內溫度與氣流。要操作氣候控制系統，請使用位於中央控制台上的按鈕與旋鈕。  溫度：溫度旋鈕用來調整車內溫度。順時針旋轉可提高溫度，逆時針旋轉則降低溫度。 氣流：氣流旋鈕控制車內氣流量。順時針旋轉可增加氣流，逆時針旋轉則減少氣流。  風扇速度：風扇速度旋鈕調整風扇轉速。順時針旋轉可加快風扇速度，逆時針旋轉則降低風速。 模式：模式按鈕允許你選擇所需模式。可用的模式有： 自動：車輛會自動調整溫度與氣流以維持舒適環境。 制冷：車輛將送出冷氣。 加熱：車輛將送出暖氣。 除霜：車輛將向擋風玻璃送暖以除霜。"
DOCUMENT2 = '你的 Google 車配備一塊大型觸控螢幕，可存取多種功能，包括導航、娛樂與氣候控制。使用觸控螢幕時，只需觸碰所需圖示。  例如，你可以觸碰「導航」圖示以獲得前往目的地的路線，或觸碰「音樂」圖示以播放你最喜愛的歌曲。'
DOCUMENT3 = "換檔  你的 Google 車採用自動變速箱。要換檔，只需將排檔桿移至所需位置。  停車：此位置用於停車，車輪將被鎖定，車輛無法移動。 倒車：此位置用於倒車。 空檔：此位置用於等候紅綠燈或交通堵塞時，車輛處於空檔狀態，除非踩下油門否則不會移動。 前進：此位置用於前進。 低速：此位置用於雪地或其他濕滑路面行駛。"

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

## 使用 ChromaDB 建立嵌入資料庫

首先，建立一個 [自訂函式](https://docs.trychroma.com/guides/embeddings#custom-embedding-functions) 用以透過 intfloat/multilingual-e5-large 生成嵌入向量。

關鍵說明：資料庫中的項目稱為「文件」，這些文件會先被寫入資料庫，稍後再檢索。查詢則是文字搜尋字串，可以是簡單關鍵字或描述所需文件的文字內容。

### 使用 intfloat/multilingual-e5-large 產生嵌入向量 (支援繁體中文)

In [4]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from sentence_transformers import SentenceTransformer

class MultilingualE5EmbeddingFunction(EmbeddingFunction):
    def __init__(self, model_name: str = "intfloat/multilingual-e5-large"):
        # 載入支援多語言的 E5 大型模型
        self.model = SentenceTransformer(model_name)

    def __call__(self, input: Documents) -> Embeddings:
        """
        根據輸入的文件產生嵌入向量
        參數：
            input (Documents): 輸入文件（字串列表）
        回傳：
            Embeddings: 對應的向量列表
        """
        # 使用模型編碼文件，並將結果轉成列表
        embeddings = self.model.encode(input, convert_to_numpy=True).tolist()
        return embeddings

In [ ]:
# 手動測試觀察編碼結果
# embed_fn = MultilingualE5EmbeddingFunction()
# embedding_results = embed_fn(documents)
# print("文件嵌入結果：", embedding_results)

現在建立一個使用 GeminiEmbeddingFunction 的 [Chroma 資料庫](https://docs.trychroma.com/getting-started) client，並把上面定義好的文件加進資料庫裡。

In [7]:
import chromadb

# 定義資料庫名稱
DB_NAME = "multilingual_db"
embed_fn = MultilingualE5EmbeddingFunction()

# 建立 ChromaDB 客戶端，並使用自訂嵌入函式
chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

# 將文件新增至資料庫（此處文件可依需求替換）
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

資料庫中文件數： 3


In [9]:
# 檢查資料庫中的文件數量
print("資料庫中文件數：", db.count())
# 你也可以預覽資料內容：
# db.peek(0)

資料庫中文件數： 3


## 檢索(Retrieval)：尋找相關文件

要在 Chroma 資料庫中進行搜尋，請呼叫 `query` 方法。注意，此時要切換至 `retrieval_query` 模式來生成查詢的嵌入向量。


In [10]:
from IPython.display import Markdown

# 使用指定查詢搜尋 Chroma 資料庫
query = "如何使用觸控螢幕播放音樂？"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

你的 Google 車配備一塊大型觸控螢幕，可存取多種功能，包括導航、娛樂與氣候控制。使用觸控螢幕時，只需觸碰所需圖示。  例如，你可以觸碰「導航」圖示以獲得前往目的地的路線，或觸碰「音樂」圖示以播放你最喜愛的歌曲。

## 增強生成(Augmented generation)：回答問題

既然你已經從文件集中檢索到一個相關段落（檢索步驟），接下來就可以組合生成提示，讓 Gemini API 根據該提示生成最終答案。請注意，在此例中僅檢索到單一段落。在實務上，尤其當底層資料集規模龐大時，你會希望檢索多個結果，並讓 Gemini 模型判斷哪些段落與問題回答最相關。因此，即使部分檢索結果與問題無直接關係也沒關係，此生成步驟會忽略不相干內容。


> 這裡我們拿 Google Gemini 作為 LLM 回答的模型。不參與檢索(Retrieval)過程。

In [14]:
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'
client = genai.Client(api_key=GOOGLE_API_KEY)

In [12]:
query_oneline = query.replace("\n", " ")

# 此提示可用來指示模型的語氣、應遵循或避免的主題等。
prompt = f"""你是一個友善且資訊豐富的機器人，請利用以下參考段落回答問題。
請務必以完整句子回覆，內容要詳盡且包含所有相關背景資訊。
但由於對象是非技術使用者，請分解複雜概念，並採用友善且口語化的語氣。如果段落與答案無關，你可以忽略它。

問題：{query_oneline}
"""

# 將檢索到的文件段落加入提示中
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"段落：{passage_oneline}\n"

print(prompt)

你是一個友善且資訊豐富的機器人，請利用以下參考段落回答問題。
請務必以完整句子回覆，內容要詳盡且包含所有相關背景資訊。
但由於對象是非技術使用者，請分解複雜概念，並採用友善且口語化的語氣。如果段落與答案無關，你可以忽略它。

問題：如何使用觸控螢幕播放音樂？
段落：你的 Google 車配備一塊大型觸控螢幕，可存取多種功能，包括導航、娛樂與氣候控制。使用觸控螢幕時，只需觸碰所需圖示。  例如，你可以觸碰「導航」圖示以獲得前往目的地的路線，或觸碰「音樂」圖示以播放你最喜愛的歌曲。



In [16]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

哈囉！想在你的 Google 車上用觸控螢幕播放音樂嗎？很簡單，就像操作手機一樣！你的車子配備了一個大大的觸控螢幕，上面有很多圖示，每個圖示都代表不同的功能。

要播放音樂，你只需要在螢幕上找到一個寫著「音樂」的圖示，然後輕輕點一下它。這樣就會打開音樂播放的介面，你就可以從裡面選擇想聽的歌曲，開始享受音樂囉！是不是很方便呢？
